<a href="https://colab.research.google.com/github/Pramsi/Can-t_tame_that_Python_Semester_Project/blob/main/Can't_tame_that_Python_Semester_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [61]:
#for data manipulation
import numpy as np
import pandas as pd

#for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Other tooling
import os

#NLP
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
!pip install --upgrade emoji
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Import Datafiles

In [ ]:
# create kaggle directory and add kaggle.json
from google.colab import files
files.upload()

! mkdir -p ~/.kaggle #create kaggle directory
! mv kaggle.json ~/.kaggle/ #move json file there
! chmod 600 ~/.kaggle/kaggle.json #change permission of the file

Saving kaggle.json to kaggle.json


In [ ]:
#create gitignore and add kaggle
%%writefile .gitignore
.kaggle/

Overwriting .gitignore


In [ ]:
#upload brexit dataset
!kaggle datasets download -d visalakshiiyer/twitter-data-brexit
! mkdir brexitTweets #create folder with files
! unzip twitter-data-brexit.zip -d brexitTweets #unzip data and save in folder

twitter-data-brexit.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘brexitTweets’: File exists
Archive:  twitter-data-brexit.zip
replace brexitTweets/TweetDataset_AntiBrexit_Jan-Mar2022.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: brexitTweets/TweetDataset_AntiBrexit_Jan-Mar2022.csv  
replace brexitTweets/TweetDataset_ProBrexit_Jan-Mar2022.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: brexitTweets/TweetDataset_ProBrexit_Jan-Mar2022.csv  


In [ ]:
#Load Datasets
dfAnti = pd.read_csv("brexitTweets/TweetDataset_AntiBrexit_Jan-Mar2022.csv")
dfPro = pd.read_csv("brexitTweets/TweetDataset_ProBrexit_Jan-Mar2022.csv")

In [ ]:
#Check Data
dfAnti.head()

,Unnamed: 0,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,...,Twitter Screen Name,User Profile Url,Twitter Bio,Twitter Followers,Twitter Following,Alternate Date Format,Time,State,City,Document Tags
0,0,09-Jan-2022 11:58PM,NaN,https://twitter.com/nickynicky77551/statuses/1...,NaN,"RT @cnapan: #JohnsonOut is now running at 218,...",Twitter,@nickynicky77551,Unknown,NaN,...,nicky@nicky7755,https://twitter.com/nickynicky77551,"Ardent Remainer; Rejoiner now, Support NHS 💙 #...",1697.0,2576.0,"Jan 9, 2022",11:58 PM,NaN,NaN,NaN
1,1,09-Jan-2022 11:58PM,NaN,https://twitter.com/lines12345/statuses/148032...,NaN,RT @Femi_Sorry: My point: Any Labour leader wh...,Twitter,@lines12345,Unknown,NaN,...,Helen,https://twitter.com/lines12345,Tory hating Remainer,11.0,92.0,"Jan 9, 2022",11:58 PM,NaN,NaN,NaN
2,2,09-Jan-2022 11:58PM,NaN,https://twitter.com/millymoo97/statuses/148032...,NaN,RT @andy_murray: QT @Nigel_Farage: Please reco...,Twitter,@millymoo97,Unknown,NaN,...,seonaid mcgill,https://twitter.com/millymoo97,"Scottish, European, anti-Brexit, pro-EU, freel...",2129.0,3880.0,"Jan 9, 2022",11:58 PM,NaN,NaN,NaN
3,3,09-Jan-2022 11:58PM,NaN,https://twitter.com/ValueSurplus/statuses/1480...,NaN,RT @mikegalsworthy: We’re the first country in...,Twitter,@valuesurplus,United Kingdom,NaN,...,Rightwing politics is a social virus #BLM 🇵🇸🌈,https://twitter.com/ValueSurplus,Classical Cultural Marxist.\nAnti Brexit estab...,2420.0,4952.0,"Jan 9, 2022",11:58 PM,NaN,NaN,NaN
4,4,09-Jan-2022 11:57PM,NaN,https://twitter.com/Jackcdawes/statuses/148032...,NaN,"RT @StevePeers: QT @andy_murray: Game, set and...",Twitter,@jackcdawes,United Kingdom,NaN,...,Jack Dawes #FBPEGlobal #JohnsonOut #RejoinEU,https://twitter.com/Jackcdawes,"Lover of crime & horror fiction, history, & Me...",18304.0,19889.0,"Jan 9, 2022",11:57 PM,England,London,NaN


Preprocessing

In [ ]:
# label pro and anti tweets and concatenate the two files
dfAnti['Position'] = 0
dfPro['Position'] = 1

df = pd.concat([dfAnti, dfPro])

#drop unwanted data
df.drop(['Unnamed: 0', 'Date', 'Headline', 'URL', 'Opening Text', 'Source', 'Influencer', 'Country', 'Subregion', 'Language', 'Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo', 'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership', 'Engagement', 'AVE', 'Key Phrases', 'Input Name', 'Keywords', 'Twitter Authority', 'Tweet Id', 'Twitter Id', 'Twitter Client', 'Twitter Screen Name', 'User Profile Url', 'Twitter Bio', 'Twitter Followers', 'Twitter Following', 'Alternate Date Format', 'Time', 'State', 'City', 'Document Tags'], axis = 1, inplace=True)

#count the data
df.Position.value_counts()

0    210574
1    147631
Name: Position, dtype: int64

In [ ]:
#Clean Hit Sentence - Remove Retweets
df_no_rt = df[df['Hit Sentence'].str.contains("RT @")==False]
df_no_qt = df_no_rt[df_no_rt['Hit Sentence'].str.contains("QT @")==False]

#how many tweets were retweeted and qoted?
print(f'Amount of retweets and quote tweets in percentage:{round(100*(1-len(df_no_qt)/len(df)),1)}%')
print(df_no_qt)

Amount of retweets and quote tweets in percentage:78.8%
                                             Hit Sentence Sentiment  Position
7       @JayneMonie1 @HowardJ91352838 @davesumnersmith...   Neutral         0
17      @MPIainDS You have made it nigh on impossible ...   Neutral         0
21      @TheGhos72578902 @SallyAnnC3 @AndieHalihan @Ch...   Neutral         0
36      @snb19692 FPTP - totally discredited! Not leas...  Negative         0
39      @pauld4dog @jarviboy1974uk @LloydHardy Superma...   Neutral         0
...                                                   ...       ...       ...
147604  @sandieshoes We did have a fight, for years, t...   Neutral         1
147610  @ExpoSpurs He’s just another pathetic sneering...  Negative         1
147613  #BorisJohnsonOut inthegardenforaparty it’s not...   Neutral         1
147626  @JamesRobvincent @7nestingwrens He was also fi...   Neutral         1
147628  Ludicrous to suggest Boris is insulting Ukrain...  Negative         1

[76098 

In [ ]:
#How many Pro and Anti Tweets are left
df_no_qt.Position.value_counts()

1    40387
0    35711
Name: Position, dtype: int64

In [ ]:
#Preprocess Helper Functions

#to lower case
def lowercase(tweet: str) -> str:
  preprocessedTweet = str.lower(tweet)
  return preprocessedTweet

#remove hashtags
def remove_hashtags(tweet: str) -> str:
  preprocessedTweet = re.sub('#[a-zA-Z0-9_]+','',tweet)
  return preprocessedTweet

#remove mentions
def remove_mentions(tweet: str) -> str:
  preprocessedTweet = re.sub('@[a-zA-Z0-9_]+','',tweet)
  return preprocessedTweet

#remove emojis
def remove_emojis(tweet: str) -> str:
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub('', tweet)

#remove stopwords
def remove_stopwords(tweet: str) -> str:
    word_list = tweet.split()
    filtered_words = [word for word in word_list if word not in set(stopwords.words('english'))]
    return ' '.join(filtered_words)

# Function to combine all preprocessing steps
def preprocess_tweet(tweet: str) -> str:
    tweet = lowercase(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_mentions(tweet)
    tweet = remove_stopwords(tweet)
    tweet = remove_emojis(tweet)
    return tweet

In [ ]:
# no retweets and no quote tweets / no cleaned column yet
df_no_qt.head()

,Hit Sentence,Sentiment,Position
7,@JayneMonie1 @HowardJ91352838 @davesumnersmith...,Neutral,0
17,@MPIainDS You have made it nigh on impossible ...,Neutral,0
21,@TheGhos72578902 @SallyAnnC3 @AndieHalihan @Ch...,Neutral,0
36,@snb19692 FPTP - totally discredited! Not leas...,Negative,0
39,@pauld4dog @jarviboy1974uk @LloydHardy Superma...,Neutral,0


In [ ]:
# create cleaned tweet column
df_no_qt['tweet_clean'] = df_no_qt['Hit Sentence'].apply(preprocess_tweet)

<ipython-input-36-7e9d578539c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_qt['tweet_clean'] = df_no_qt['Hit Sentence'].apply(preprocess_tweet)


In [ ]:
# with cleaned tweet column
df_no_qt.head()

,Hit Sentence,Sentiment,Position,tweet_clean
7,@JayneMonie1 @HowardJ91352838 @davesumnersmith...,Neutral,0,"money’s sunak, (with gove & cummings) behind p..."
17,@MPIainDS You have made it nigh on impossible ...,Neutral,0,made nigh impossible share european funding st...
21,@TheGhos72578902 @SallyAnnC3 @AndieHalihan @Ch...,Neutral,0,haven’t wrong yet. said would lose. did. said ...
36,@snb19692 FPTP - totally discredited! Not leas...,Negative,0,fptp - totally discredited! least derisory 17....
39,@pauld4dog @jarviboy1974uk @LloydHardy Superma...,Neutral,0,supermarkets main brexit supporters quiet. bre...


Split and Vectorize the data

In [46]:
# Set X and y label
X_preprocessed = df_no_qt["tweet_clean"]
y = df_no_qt["Sentiment"]

In [62]:
# Encode the y labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [63]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_encoded, test_size=0.3, random_state=42)

In [64]:
#Vectorize the data
tfidf_vectorizer = TfidfVectorizer(stop_words='english') #make sure that all stopwords are excluded, stopword list might differ
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

count_vectorizer = CountVectorizer(stop_words="english") #make sure that all stopwords are excluded
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

Train Models with Tfidf Vectorizer

In [65]:
# Initialize a Support Vector Classifier (SVC) with a linear kernel
svm_linear = SVC(kernel='linear', C=1)

# Train the classifier using the tfidf training data and training labels
svm_linear.fit(tfidf_train, y_train)

SVC(C=1, kernel='linear')

In [66]:
# Initialize a Logistic Regression Model
log_reg = LogisticRegression(C=1, solver='lbfgs', max_iter=1000)

#Train the Linear Regression Model using the tfidf data
log_reg.fit(tfidf_train, y_train)

LogisticRegression(C=1, max_iter=1000)

In [67]:
# Initialize a Random Forest Classifier
random_forest = RandomForestClassifier(max_depth=None, n_estimators=100, random_state=42)

# Fit the Random Forest Classifier on the training data
random_forest.fit(tfidf_train, y_train)

RandomForestClassifier(random_state=42)

In [68]:
# Initialize Naive Bayes Classifier
nb_classifier = MultinomialNB()

# Fit the Naive Bayes Classifier with the tfidf data
nb_classifier.fit(tfidf_train, y_train)

MultinomialNB()

Make predictions with Tfidf vectorized data

In [69]:
# Predict the labels for the tfidf vectorized data using a linear SVM model
y_pred_svm_tfidf = svm_linear.predict(tfidf_test)

# Predict the labels for the tfidf vectorized data using a logistic regression model
y_pred_logreg_tfidf = log_reg.predict(tfidf_test)

# Predict the labels for the tfidf vectorized data using a Random Forest Classifier
y_pred_forest_tfidf = random_forest.predict(tfidf_test)

# Predict the labels for the tfidf vectorized data using a Naive Bayes Classifier
y_pred_nb_tfidf = nb_classifier.predict(tfidf_test)

Evaluate our Models with the Tfidf vectorized data

In [79]:
#SVM Evaluation
print(f"SVM Evaluation:")
# Calculate and print the accuracy score
accuracy_svm_tfidf = accuracy_score(y_test, y_pred_svm_tfidf)
print(f"Accuracy: {accuracy_svm_tfidf}")

# Calculate and print the F1 score
f1_svm_tfidf = f1_score(y_test, y_pred_svm_tfidf, average='macro')
print(f"F1 Score: {f1_svm_tfidf}")

# Calculate and print the precision score
precision_svm_tfidf = precision_score(y_test, y_pred_svm_tfidf, average='macro')
print(f"Precision: {precision_svm_tfidf}")

# Calculate and print the recall score
recall_svm_tfidf = recall_score(y_test, y_pred_svm_tfidf, average='macro')
print(f"Recall: {recall_svm_tfidf}")

print(f"---------------------------------------------")

print(f"Logistic Regression Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_logreg_tfidf = accuracy_score(y_test, y_pred_logreg_tfidf)
print(f"Accuracy: {accuracy_logreg_tfidf}")

# Calculate and print the F1 score
f1_logreg_tfidf = f1_score(y_test, y_pred_logreg_tfidf, average='macro')
print(f"F1 Score: {f1_logreg_tfidf}")

# Calculate and print the precision score
precision_logreg_tfidf = precision_score(y_test, y_pred_logreg_tfidf, average='macro')
print(f"Precision: {precision_logreg_tfidf}")

# Calculate and print the recall score
recall_logreg_tfidf = recall_score(y_test, y_pred_logreg_tfidf, average='macro')
print(f"Recall: {recall_logreg_tfidf}")

print(f"---------------------------------------------")

print(f"Random Forest Evaluation:")
#Random Forest Evaluation
# Calculate and print the accuracy score
accuracy_forest_tfidf = accuracy_score(y_test, y_pred_forest_tfidf)
print(f"Accuracy: {accuracy_forest_tfidf}")

# Calculate and print the F1 score
f1_forest_tfidf = f1_score(y_test, y_pred_forest_tfidf, average='macro')
print(f"F1 Score: {f1_forest_tfidf}")

# Calculate and print the precision score
precision_forest_tfidf = precision_score(y_test, y_pred_forest_tfidf, average='macro')
print(f"Precision: {precision_forest_tfidf}")

# Calculate and print the recall score
recall_forest_tfidf = recall_score(y_test, y_pred_forest_tfidf, average='macro')
print(f"Recall: {recall_forest_tfidf}")

print(f"---------------------------------------------")

print(f"Naive Bayes Classifier Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_nb_tfidf = accuracy_score(y_test, y_pred_nb_tfidf)
print(f"Accuracy: {accuracy_nb_tfidf}")

# Calculate and print the F1 score
f1_nb_tfidf = f1_score(y_test, y_pred_nb_tfidf, average='macro')
print(f"F1 Score: {f1_nb_tfidf}")

# Calculate and print the precision score
precision_nb_tfidf = precision_score(y_test, y_pred_nb_tfidf, average='macro', zero_division=1)
print(f"Precision: {precision_nb_tfidf}")

# Calculate and print the recall score
recall_nb_tfidf = recall_score(y_test, y_pred_nb_tfidf, average='macro', zero_division=1)
print(f"Recall: {recall_nb_tfidf}")

print(f"---------------------------------------------")

SVM Evaluation:
Accuracy: 0.7622426631625054
F1 Score: 0.6577834285420475
Precision: 0.7375067307549591
Recall: 0.6110662524583587
---------------------------------------------
Logistic Regression Evaluation:
Accuracy: 0.7602715724923347
F1 Score: 0.646498540600515
Precision: 0.7311587309372549
Recall: 0.6008915527768459
---------------------------------------------
Random Forest Evaluation:
Accuracy: 0.7378887428821725
F1 Score: 0.6297539477302461
Precision: 0.6905393407190975
Recall: 0.5946839026812307
---------------------------------------------
Naive Bayes Classifier Evaluation:
Accuracy: 0.6781865965834428
F1 Score: 0.3652524100550634
Precision: 0.8520927703134421
Recall: 0.36301792602098854
---------------------------------------------
